In [48]:
import psycopg2 as pg
import json

In [82]:
def runQuery(query, con, response=False):
    try:
        cur = con.cursor()
        cur.execute(query)
        if response:
            cols = [desc[0] for desc in cur.description]
            rows = cur.fetchall()
        cur.close()
        if response:
            return rows, cols
        else: return
    except (Exception, pg.DatabaseError) as error:
        print(error)

In [191]:
con = pg.connect(host='mydatabase.c2z535twmu4p.us-west-2.rds.amazonaws.com',
                 database='postgres',
                 user='postgres',
                 password='password',
                 port = '5432',
                 connect_timeout=3,
                )

create_game_table = '''
CREATE TABLE IF NOT EXISTS games (
    game_id varchar(32) NOT NULL PRIMARY KEY,
    game_state json NOT NULL
);
'''

create_user_table = '''
CREATE TABLE IF NOT EXISTS users (
    user_id varchar(32) NOT NULL PRIMARY KEY,
    user_info json
);
'''

create_session_table = '''
CREATE TABLE IF NOT EXISTS sessions (
    game_id varchar(32) NOT NULL,
    user_id varchar(32) NOT NULL,
    FOREIGN KEY (game_id) REFERENCES games(game_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE ON UPDATE CASCADE,
    PRIMARY KEY (game_id, user_id)
);
CREATE INDEX game_id on sessions(game_id);
'''
runQuery(create_game_table, con)
runQuery(create_user_table, con)
runQuery(create_session_table, con)


In [192]:
runQuery('SELECT * from sessions', con, response=True)

([], ['game_id', 'user_id'])

In [193]:
def sql_insert(table, columns, rows):
    return f'''
    INSERT INTO {table} ({','.join(columns)})
    VALUES {','.join(['(' + ','.join([f"'{cell}'" for cell in row]) + ')' for row in rows])}
    '''

def sql_select(table, columns):
    return f"SELECT {','.join(columns)} from {table}"

In [194]:
runQuery(sql_insert('users', ['user_id'], [[x] for x in ['aUser','Mike','Tom']]), con)
runQuery(sql_select('users','*'), con, response=True)

([('aUser', None), ('Mike', None), ('Tom', None)], ['user_id', 'user_info'])

In [195]:
runQuery(sql_insert('games', ['game_id', 'game_state'], [[x, json.dumps({})] for x in ['aGame','1234','abcd']]), con)
runQuery(sql_select('games','*'), con, response=True)

([('aGame', {}), ('1234', {}), ('abcd', {})], ['game_id', 'game_state'])

In [196]:
runQuery(sql_insert('sessions', ['user_id', 'game_id'], [['aUser', x] for x in ['aGame','1234','abcd']]), con)
runQuery(sql_select('sessions','*'), con, response=True)

([('aGame', 'aUser'), ('1234', 'aUser'), ('abcd', 'aUser')],
 ['game_id', 'user_id'])

In [197]:
runQuery('''
SELECT sessions.user_id, game_id from sessions
INNER JOIN users on users.user_id=sessions.user_id
''', con, response=True)

([('aUser', 'aGame'), ('aUser', '1234'), ('aUser', 'abcd')],
 ['user_id', 'game_id'])

In [100]:
con.close()

In [74]:
con.commit()